In [49]:
# @title 1. Install & Setup
import os
import json
import time
from elevenlabs import ElevenLabs, VoiceSettings

# Create necessary directories
OUTPUT_DIR = 'output'
OUTPUT_AUDIO_ELEVEN_SUBDIR = 'audio/eleven_flash'
full_output_eleven_path = os.path.join(OUTPUT_DIR, OUTPUT_AUDIO_ELEVEN_SUBDIR)
os.makedirs(full_output_eleven_path, exist_ok=True)

print("✅ Setup complete. Output folder: ", full_output_eleven_path)

✅ Setup complete. Output folder:  output/audio/eleven_flash


In [50]:
# @title 2. Configuration

# 🔑 REPLACE THIS WITH YOUR ACTUAL ELEVENLABS API KEY
ELEVEN_API_KEY = "sk_da55ff02071806fe7e5195dec222dcfb8c3354e7fdbe60b2"

INPUT_JSON_FILE = 'manifest.json'
AUDIO_WAV_DIR = 'audio_wav'
FOX_TEXT = "The quick brown fox jumps over the lazy dog. But what about the man who watched over them? He barely noticed."

print("✅ Configuration set.")

✅ Configuration set.


In [51]:
# @title 3. ElevenLabs Transient Voice Logic (Fixed for SDK v2.x)

def generate_eleven_audio_transient(user_id, reference_audio_path, original_text, fox_text):
    """
    Transient Voice Cloning Flow (Updated for SDK v2.x):
    1. Add Voice via client.voices.ivc.create
    2. Generate Original TTS via client.text_to_speech.convert
    3. Generate Fox TTS via client.text_to_speech.convert
    4. Delete Voice via client.voices.delete
    """
    client = ElevenLabs(api_key=ELEVEN_API_KEY)
    voice_name = f"3min-test-{user_id}"
    voice_id = None
    
    try:
        # 1. Add Voice (Instant Voice Cloning)
        print(f"   📤 Uploading reference for {user_id}...")
        if not os.path.exists(reference_audio_path):
            print(f"   ❌ Reference file missing: {reference_audio_path}")
            return False
            
        with open(reference_audio_path, "rb") as f:
            # SDK v2.x uses client.voices.ivc.create
            voice = client.voices.ivc.create(
                name=voice_name,
                files=[f],
                description=f"Temporary clone for {user_id}"
            )
        voice_id = voice.voice_id
        print(f"   ✅ Voice Created: {voice_id} (Name: {voice_name})")
        
        # 2. Generate Original
        out_orig = os.path.join(full_output_eleven_path, f"spk-{user_id}-eleven-original.mp3")
        print(f"   🎙️ Generating Original Text...")
        audio_orig_gen = client.text_to_speech.convert(
            voice_id=voice_id,
            text=original_text,
            model_id="eleven_flash_v2_5"
        )
        with open(out_orig, "wb") as f:
            for chunk in audio_orig_gen:
                f.write(chunk)
        
        # 3. Generate Fox
        out_new = os.path.join(full_output_eleven_path, f"spk-{user_id}-eleven-new.mp3")
        print(f"   🎙️ Generating Fox Text...")
        audio_new_gen = client.text_to_speech.convert(
            voice_id=voice_id,
            text=fox_text,
            model_id="eleven_flash_v2_5"
        )
        with open(out_new, "wb") as f:
            for chunk in audio_new_gen:
                f.write(chunk)
                
        print(f"   ✅ Files saved: {os.path.basename(out_orig)}, {os.path.basename(out_new)}")
        return True
        
    except Exception as e:
        print(f"   ❌ ElevenLabs Error: {e}")
        return False
        
    finally:
        # 4. Delete Voice with Safety Guard
        if voice_id:
            try:
                # 🛡️ Safety Guard: Verify name before deletion
                voice_info = client.voices.get(voice_id)
                if voice_info.name == voice_name:
                    print(f"   🗑️ Safety Verified: Deleting temporary voice '{voice_name}' ({voice_id})...")
                    client.voices.delete(voice_id)
                    print("   ✅ Voice Deleted.")
                else:
                    print(f"   ⚠️ SAFETY ABORT: Voice name mismatch. Skipping deletion.")
            except Exception as de:
                print(f"   ⚠️ Could not carry out safety-check or delete voice {voice_id}: {de}")

print("✅ ElevenLabs generation logic defined (Transient Flow with Safety Guard).")

✅ ElevenLabs generation logic defined (Transient Flow with Safety Guard).


In [52]:
# @title 4. Run Batch Generation (All 120 Speakers)

def run_batch_generation():
    if ELEVEN_API_KEY == "YOUR_ELEVENLABS_API_KEY":
        print("❌ ERROR: Please set your ELEVEN_API_KEY in Section 2 first!")
        return

    with open(INPUT_JSON_FILE, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    samples = data['samples']
    total = len(samples)
    print(f"🚀 Starting mass generation for {total} speakers...")
    
    success_count = 0
    
    for i, sample in enumerate(samples):
        user_id = sample['user_id']
        text = sample['text']
        
        # Construct reference path from audio_wav
        wav_filename = os.path.basename(sample['audio_file']).replace('.webm', '.wav').replace('.m4a', '.wav').replace('.ogg', '.wav')
        ref_path = os.path.join(AUDIO_WAV_DIR, wav_filename)
        
        # Check if already exists (optional skip logic)
        out_orig = os.path.join(full_output_eleven_path, f"spk-{user_id}-eleven-original.mp3")
        if os.path.exists(out_orig):
            print(f"⏭️ [{i+1}/{total}] Skipping {user_id} (Already exists)")
            success_count += 1
            continue
            
        print(f"🎙️ [{i+1}/{total}] User: {user_id}")
        success = generate_eleven_audio_transient(user_id, ref_path, text, FOX_TEXT)
        
        if success:
            success_count += 1
        else:
            print(f"   ❌ FAILED: {user_id}")
            
        # Small delay to be gentle on API
        time.sleep(1)
        
    print(f"\n🏆 Batch Generation Complete! Success: {success_count}/{total}")

# Uncomment to run batch in notebook
run_batch_generation()

🚀 Starting mass generation for 30 speakers...
🎙️ [1/30] User: 153293
   📤 Uploading reference for 153293...
   ✅ Voice Created: GkqIh9HFPbAVEzOAyt4H (Name: 3min-test-153293)
   🎙️ Generating Original Text...
   🎙️ Generating Fox Text...
   ✅ Files saved: spk-153293-eleven-original.mp3, spk-153293-eleven-new.mp3
   🗑️ Safety Verified: Deleting temporary voice '3min-test-153293' (GkqIh9HFPbAVEzOAyt4H)...
   ✅ Voice Deleted.
🎙️ [2/30] User: 185885
   📤 Uploading reference for 185885...
   ✅ Voice Created: LbXPIFv6wDHrLG1ZLnVw (Name: 3min-test-185885)
   🎙️ Generating Original Text...
   🎙️ Generating Fox Text...
   ✅ Files saved: spk-185885-eleven-original.mp3, spk-185885-eleven-new.mp3
   🗑️ Safety Verified: Deleting temporary voice '3min-test-185885' (LbXPIFv6wDHrLG1ZLnVw)...
   ✅ Voice Deleted.
🎙️ [3/30] User: 173685
   📤 Uploading reference for 173685...
   ✅ Voice Created: fdAmqZ8QwED2iDZ0KIRQ (Name: 3min-test-173685)
   🎙️ Generating Original Text...
   🎙️ Generating Fox Text...
   

In [54]:
# @title 5. Update Metadata in speakers.ts

def update_eleven_metadata():
    ts_path = os.path.join(OUTPUT_DIR, 'speakers.ts')

    if not os.path.exists(ts_path):
        print(f"❌ {ts_path} not found.")
        return

    with open(ts_path, 'r', encoding='utf-8') as f:
        content = f.read()

    # Extract JSON part
    equals_idx = content.find('=')
    start_idx = content.find('[', equals_idx)
    end_idx = content.rfind(']') + 1

    if equals_idx == -1 or start_idx == -1 or end_idx == 0:
        print("❌ Could not parse JSON from speakers.ts")
        return

    json_str = content[start_idx:end_idx]
    try:
        speakers_data = json.loads(json_str)
    except json.JSONDecodeError as e:
        print(f"❌ JSON parse error: {e}")
        return

    print(f"📝 Updating metadata for {len(speakers_data)} speakers...")
    update_count = 0

    for speaker in speakers_data:
        user_id = speaker['id'].replace('spk-', '')

        # Check if ElevenLabs files exist
        orig_filename = f"spk-{user_id}-eleven-original.mp3"
        new_filename = f"spk-{user_id}-eleven-new.mp3"

        path_orig = os.path.join(full_output_eleven_path, orig_filename)
        path_new = os.path.join(full_output_eleven_path, new_filename)

        if os.path.exists(path_orig) and os.path.exists(path_new):
            # Check if model already exists (eleven-flash)
            has_flash = any(m['modelId'] == 'eleven-flash' for m in speaker['models'])

            model_entry = {
                "modelId": "eleven-flash",
                "modelName": "eleven (flash)",
                "clonedOriginalAudio": f"/{OUTPUT_AUDIO_ELEVEN_SUBDIR}/{orig_filename}",
                "clonedOriginalText": speaker['originalText'],
                "clonedNewAudio": f"/{OUTPUT_AUDIO_ELEVEN_SUBDIR}/{new_filename}",
                "clonedNewText": FOX_TEXT
            }

            if not has_flash:
                speaker['models'].append(model_entry)
                update_count += 1
            else:
                # Update existing
                for m in speaker['models']:
                    if m['modelId'] == 'eleven-flash':
                        m.update(model_entry)

    # Write back
    new_content = f"export const speakers: Speaker[] = {json.dumps(speakers_data, indent=2)};\n"

    with open(ts_path, 'w', encoding='utf-8') as f:
        f.write(new_content)

    print(f"✅ Updated {update_count} speakers with ElevenLabs Flash data in {ts_path}")

update_eleven_metadata()

📝 Updating metadata for 150 speakers...
✅ Updated 0 speakers with ElevenLabs Flash data in output/speakers.ts
